# Код Славы, парсинг Возраста, Роста, Фото


Сначала введем основу, то есть импорт библиотек и самого сайта

In [43]:
# Что, начнем проект. Я очень ленивый негодяй, но тем не менее, попробую сделать.
# Сначала печатаем библиотеки, который нам нужны


from urllib.request import urlopen as urll  # Этой строкой посылаем запрос на возможность получения url страниц.Request обязательна, она дает возможность получить страницу в Интернете
from bs4 import BeautifulSoup  # bs4 содержит не только суп, поэтому необходимо именно его загрузить.Сам суп даёт возможность работать с html файлами, то есть превращать более читабельнй вид
import re
import requests
import pandas as pd

page = urll('https://lizaalert.org/forum/viewtopic.php?f=173&t=24632')
soup = BeautifulSoup(page)


In [44]:
df = pd.DataFrame({
    'Возраст' : [],
    'Рост' : [],
    'Фото': [],
    'Пол' : [],
    'Жив/Мертв' : []
     })
df

,Возраст,Рост,Фото,Пол,Жив/Мертв


In [45]:
    allinfo = soup.findAll('div',class_= "content")

In [46]:
    readytext = allinfo[0].text
    readytext

'Левашова Валентина Ивановна 80 лет, г. Невинномысск 26 февраля 2019 около 17-15 ушла из дома и не вернуласьПриметы:Рост: 154 см.Телосложение: нормальноеЦвет глаз: зелёныеВолосы: седые, короткая стрижка, крашенныеБыла одета: светло-серая куртка с капюшоном на молнии, черные утепленные штаны, черный берет, черные угги.С собой: ничегоТаблица прозвона больниц и ОВДОриентировка на печатьОриентировка на репост--------------------------------------------------СБОР: 44.616135, 41.917763Координатор: Войдан Инфорг: Анка 8 (963) 382-30-30'

In [47]:
# Первое это было просто вход сайт и парс всего с него(точнее не сайт, а определенной анкеты)
# Теперь начинаем по анкете находить Возраст, Рост, Фото, Жив/Мертв и Пол


In [48]:
# Но начну с ВОЗРАСТА(хотя он уже запарсен, я просто повторил код Вани).То есть за предмет парсинга я беру readytext!

# Парсинг Возраста

In [70]:
    age = re.search(r'\d+ ',readytext)

In [71]:
    age =int(age.group(0)) # эТО уже у нас число)) Ура!!!!
    if age < 100:
        print(age)
    else:
        age = 2019 - age
        print(age)

80


In [51]:
# Теперь перейдем к РОСТУ ВОууууу, смог сделать, огонь!

# Парсинг Роста

In [52]:
    readytext

'Левашова Валентина Ивановна 80 лет, г. Невинномысск 26 февраля 2019 около 17-15 ушла из дома и не вернуласьПриметы:Рост: 154 см.Телосложение: нормальноеЦвет глаз: зелёныеВолосы: седые, короткая стрижка, крашенныеБыла одета: светло-серая куртка с капюшоном на молнии, черные утепленные штаны, черный берет, черные угги.С собой: ничегоТаблица прозвона больниц и ОВДОриентировка на печатьОриентировка на репост--------------------------------------------------СБОР: 44.616135, 41.917763Координатор: Войдан Инфорг: Анка 8 (963) 382-30-30'

In [53]:
    high = re.search(r'[Рр]ост \d{3}|[Рр]оста \d{3}|[Рр]оста\d{3}|[Рр]ост: \d{3}',readytext )
    high = str(high.group(0))
    high

'Рост: 154'

In [54]:
    height = re.findall(r'\d+',high)
    height = int(" ".join(height))
    print(height)

154


In [55]:
# А теперь настал момент найти ...... барабанная дробь ....... ФОТО вот, я понял, что здесь не смогу искать через readytext,
# так как он не отражает наличие фотки. И будет 2 случая - когда это фотка и когда это расклейка по объявлении


# Парсинг Фото

In [56]:
    photored = allinfo[0] # Для фото там поменяется то, что это также readytext, но без текста. Поэтому я создам новую переменную 
# photored

In [57]:
    photo = photored.img

In [58]:
    photo = str(photo)

In [59]:
    yphoto = re.findall(r'[h]ttp',photo) # Заметил косяк, нашел одну анкету, где фотки прикреплены по-другому.  
    yphoto = str(yphoto)                # Но это всего одна анкета,поэтому пока считаю можно не исправлять
    print(yphoto)
    if yphoto == "['http']":
        yphoto = "+"
    else:
        yphoto = "-";
    print(yphoto)


['http']
+


In [60]:
# Ха, есть, нашёл!!! ИЗИИИИ Катка!!!

# Парсинг Пола


In [61]:
# Для этого воспользуемся уловкой - попробуем парсить по словам ушел, ушла, пропал, пропала и т.д. Обратимся к всем изестному 
# readytext, так как он готов и с него и будем парсить

In [62]:
    readytext

'Левашова Валентина Ивановна 80 лет, г. Невинномысск 26 февраля 2019 около 17-15 ушла из дома и не вернуласьПриметы:Рост: 154 см.Телосложение: нормальноеЦвет глаз: зелёныеВолосы: седые, короткая стрижка, крашенныеБыла одета: светло-серая куртка с капюшоном на молнии, черные утепленные штаны, черный берет, черные угги.С собой: ничегоТаблица прозвона больниц и ОВДОриентировка на печатьОриентировка на репост--------------------------------------------------СБОР: 44.616135, 41.917763Координатор: Войдан Инфорг: Анка 8 (963) 382-30-30'

In [63]:
    sex = re.search(r'[Вв]ышел|[Вв]ышла|[Уу]шла|[Уу]шёл|[Уу]шел|[Пп]ропал|[Пп]ропала|[Нн]айден|[Нн]айдена|[Жж]ива|[Жж]ив', readytext)
    sex = str(sex.group(0))
    print(sex)
    if sex == "вышел" or sex == "Вышел" or sex == "ушел" or sex == "Ушел" or sex == 'ушёл' or sex == 'пропал' or sex == 'Пропал' or sex == 'Найден' or sex == 'найден' or sex == 'ушел'or sex == 'пропал'or sex == 'найден'or sex == 'Найден'or sex == 'вышел' or sex == 'Жив'or sex == 'жив':
        sex = "М"
    else:
        sex = 'Ж'

    #else:
        #sex = "Error";#"'ушла'" or "'найдена'" or "'пропала'" or 'Жива' or "'жива'"
    print(sex)

ушла
Ж


# Так,  а теперь  надо последнее Мерт или Жив!!!

In [64]:
# Все, я красавчик, все запарсил, теперь только остался Жив или Мертв. А это тоже сделать изи, через буквы, как на мужчину и женщину

In [74]:
soup.title.text # Делаем не через реадитекст, так как там не всегда указано, жив или мертв. Делаем через суп.титл

'Погибла. Левашова Валентина Ивановна 80 лет, г. Невинномысск • Форум lizaalert.org'

In [75]:
    sit = re.search(r'[Жж]ива|[Жж]ив|[Пп]огиб|[Пп]огибла|[Нн]айден|[Нн]айдена',soup.title.text)

In [76]:
    sit = str(sit.group(0))
    print(sit)
    if sit == 'Жива' or sit == 'Жив' or sit == 'жива' or sit == 'жив' or sit == 'Найден' or sit == 'найден' or sit == 'найдена'or sit == 'Найдена':
        sit = '+'
    elif sit == 'Погиб' or sit == 'Погибла'  or sit == 'Мертв' or sit == 'Мертва' or sit == 'погибла' or sit == 'погиб':
        sit = '-'
    else:
        sit = 'Ищут'
    print(sit)

Погиб
-


In [72]:
    newrow = pd.DataFrame({
        'Возраст' : [age],
        'Рост' : [height],
        'Фото': [yphoto],
        'Пол' : [sex],
        'Жив/Мертв' : [sit]
         })
    df = df.append(newrow)

df


,Возраст,Рост,Фото,Пол,Жив/Мертв
0,80.0,154.0,+,Ж,-
